In [2]:
# Read the phenotype info

import pandas as pd

csvname = '/run/media/anton/AntonData1/UKB_MRI_data_11092022/UKB_Phenotypes/project3_Anton/DataSet_11162022.csv'
df = pd.read_csv(csvname)
df

,f.eid,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.21022.0.0,f.22001.0.0,f.25750.2.0,f.25750.3.0,f.25751.2.0,f.25751.3.0,f.25752.2.0,f.25752.3.0,f.25753.2.0,f.25753.3.0,f.25754.2.0,f.25754.3.0,f.25755.2.0,f.25755.3.0
0,1000010,50.0,NaN,NaN,NaN,50.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000028,52.0,NaN,NaN,NaN,52.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000034,56.0,NaN,NaN,NaN,56.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000045,43.0,NaN,NaN,NaN,43.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000052,61.0,NaN,NaN,NaN,61.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502481,6024820,68.0,NaN,NaN,NaN,68.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502482,6024833,61.0,NaN,NaN,NaN,61.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502483,6024847,64.0,NaN,NaN,NaN,64.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502484,6024859,60.0,NaN,NaN,NaN,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Get full FC or PC matrices

import numpy as np
from pathlib import Path
import re

scan = 2
typ = 'partial'
basedir = f'/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/UKB/{typ}/'

# fc = dict()
sex = np.zeros(39652)
x = np.zeros((39652, 34716))
j = 0

for i,p in enumerate(Path(basedir).iterdir()):
    match = re.match(f'(\d+)_task-(\d+)-(\d)_{typ}.npy', p.name)
    sub = int(match.group(1))
    scn = int(match.group(3))
    if scn != scan:
        continue
#     fc[sub] = np.load(str(p))
    try:
        sex[j] = int(df[df['f.eid'] == sub]['f.22001.0.0'])
    except:
        continue
    x[j] = np.load(str(p))
    j += 1
    if i % 1000 == 0:
        print(f'Done {i}')
    
# print(len(fc))
print(x.shape, sex.shape)

Done 0
Done 1000
Done 2000
Done 3000
Done 4000
Done 5000
Done 6000
Done 7000
Done 8000
Done 9000
Done 10000
Done 11000
Done 12000
Done 14000
Done 15000
Done 17000
Done 18000
Done 19000
Done 20000
Done 21000
Done 23000
Done 24000
Done 25000
Done 26000
Done 27000
Done 28000
Done 29000
Done 30000
Done 31000
Done 32000
Done 33000
Done 34000
Done 35000
Done 36000
Done 37000
Done 38000
Done 39000
Done 40000
Done 41000
Done 42000
Done 43000
(39652, 34716) (39652,)


In [68]:
# Get ICA25 (21) or ICA100 (55) FC or PC 

import pickle

field = 25753
scan = 2

basedir = '/run/media/anton/AntonData1/UKB_MRI_data_11092022/'

fc = pickle.load(open(f'{basedir}/{field}_{scan}.pkl', 'rb'))

print(len(fc))

40471


In [3]:
from math import isnan
import numpy as np

ages = []
sex = []
x = []

for i,s in enumerate(fc.keys()):
    try:
        sex.append(int(df[df['f.eid'] == s]['f.22001.0.0']))
        ages.append(float(df[df['f.eid'] == s]['f.21003.2.0']))
        x.append(fc[s])
    except:
        continue

ages = np.array(ages)
sex = np.array(sex)
x = np.stack(x)

print(x.shape, ages.shape, sex.shape)

(39652, 34716) (39652,) (39652,)


In [14]:
# Try MLP with batch size

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(34716,100).float().cuda()
        self.fc2 = nn.Linear(100,2).float().cuda()
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x.squeeze()
    
nepochs = 1000
pperiod = 50
bsize = 5000
errs = []
nullerrs = []

N = 15000
M = 15000

for rep in range(4):
    mlp = MLP()
    optim = torch.optim.Adam(mlp.parameters(), lr=1e-4, weight_decay=1e-4)
    celoss = nn.CrossEntropyLoss()
    
    perm = np.random.permutation(M)

    xtr, xt, ytr, yt = train_test_split(x[perm], sex[perm], stratify=sex[perm], train_size=0.8)

    for i in range(nepochs):
        sz = min(bsize,N)
        for bs in range(0,N,sz):
            xb = torch.from_numpy(xtr[bs:bs+sz]).float().cuda()
            yb = torch.from_numpy(ytr[bs:bs+sz]).long().cuda()
            yhat = mlp(xb)
            loss = celoss(yhat, yb)
            loss.backward()
            optim.step()
            optim.zero_grad()
            if i % pperiod == 0 or i == nepochs-1:
                print(f'{i} {float(loss)}')
        if float(loss) < 0.06:
            break

    print('Complete')

    xb = torch.from_numpy(xt).float().cuda()
    yb = torch.from_numpy(yt).long().cuda()
    with torch.no_grad():
        yhat = mlp(xb)
        err = torch.mean((torch.argmax(yhat, dim=1) == yb).float())
        errs.append(err.detach().cpu().numpy())
    nullerr = np.mean(ytr == 0)
    if nullerr < 0.5:
        nullerr = 1-nullerr
    nullerrs.append(nullerr)
    print(err, nullerr)
        
print(np.mean(errs), np.std(errs))
print(np.mean(nullerrs), np.std(nullerrs))

0 0.6929731965065002
0 0.6909469962120056
0 0.6890694499015808
50 0.21767772734165192
50 0.2164025902748108
50 0.13930538296699524
Complete
tensor(0.9003, device='cuda:0') 0.5293333333333333
0 0.6918989419937134
0 0.6893101334571838
0 0.6860581636428833
50 0.19293081760406494
50 0.19581303000450134
50 0.1195017620921135
Complete
tensor(0.9067, device='cuda:0') 0.5293333333333333
0 0.6907467842102051
0 0.6903534531593323
0 0.6874201893806458
50 0.2075042426586151
50 0.20515090227127075
50 0.1262725591659546
Complete
tensor(0.9047, device='cuda:0') 0.5293333333333333
0 0.696287214756012
0 0.6933346390724182
0 0.6909850239753723
50 0.20837169885635376
50 0.2052789032459259
50 0.12866611778736115
Complete
tensor(0.9123, device='cuda:0') 0.5293333333333333
0.90599996 0.0043140464
0.5293333333333333 0.0
